# Preprocessing

## Library

In [ ]:
# !git clone https://github.com/ssut/py-hanspell.git
# !pip install kss

In [1]:
# 수정한 hanspell 불러오기
from py_hanspell.hanspell import spell_checker
from py_hanspell.hanspell.constants import CheckResult

import numpy as np
import pandas as pd
import kss
import re
from tqdm.notebook import tqdm
import os

import warnings
warnings.filterwarnings("ignore")

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

## Path

In [2]:
base_path = '../Data/Base/'
save_path = '../Data/QnA/'

## Load Data

In [3]:
# qna 파일 불러오기
df = pd.read_csv(base_path+'qna_table.csv')

In [4]:
df[:1]

,product_id,product_name,product_option,user_id,user_buyer,is_secret,label,question,question_time,answer,answer_time
0,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 우드,3080121,False,False,상품,어떤 유리인가요?,2021-03-19T01:16:01.000+09:00,None,None


## 비밀글 & 답변 없음 제거

In [5]:
# 전체 개수
print(df.shape)

# 비밀글 제거 > 즉, 둘 중 하나라도 비밀글이면 drop
df_remove = df[(df['question']!='비밀글입니다.')&(df['answer']!='비밀글입니다.')]

# 제거한 후 개수
print(df_remove.shape)

# 답변 없음 제거
df_remove_ = df_remove[df_remove['answer']!='None']

# 제거한 후 개수
print(df_remove_.shape)

df_remove_[:1]

(125543, 11)
(95340, 11)
(95305, 11)


,product_id,product_name,product_option,user_id,user_buyer,is_secret,label,question,question_time,answer,answer_time
3,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 우드,3077236,True,False,환불,상부 연결 목재 중 하나가 나사산이 덜 파져 있네요;; 조립하다 목재가 갈라져버립니...,2021-03-18T16:52:34.000+09:00,안녕하세요 고객님 먼데이하우스입니다.\r\n불편을 드려 죄송합니다.\r\n유선상 연...,2021-03-18T18:19:46.000+09:00


## 내용 중복 제거

In [6]:
print('질문 unique:', len(df_remove_['question'].unique()))
print('답변 unique:', len(df_remove_['answer'].unique()))

# 중복 제거
tmp_drop = df_remove_.drop_duplicates(['question','answer'], keep = 'first')
tmp_drop.reset_index(inplace=True, drop=True)

# 개수 확인하기
count = tmp_drop[['question','answer']].value_counts()
print('전체 개수: ', tmp_drop.shape)
print('중복이 없는 개수: ', len(count[count.values==1]))
tmp_drop[:1]

질문 unique: 92984
답변 unique: 80645
전체 개수:  (95190, 11)
중복이 없는 개수:  95190


,product_id,product_name,product_option,user_id,user_buyer,is_secret,label,question,question_time,answer,answer_time
0,388715,순수원목 A사이드테이블 3colors,A사이드테이블 / 우드,3077236,True,False,환불,상부 연결 목재 중 하나가 나사산이 덜 파져 있네요;; 조립하다 목재가 갈라져버립니...,2021-03-18T16:52:34.000+09:00,안녕하세요 고객님 먼데이하우스입니다.\r\n불편을 드려 죄송합니다.\r\n유선상 연...,2021-03-18T18:19:46.000+09:00


## 전처리

* 한글&숫자만 남기고 제거
* hanspell 맞춤법 검사 진행

In [7]:
df = tmp_drop

que = df['question'].to_numpy()
ans = df['answer'].to_numpy()

### 한글&숫자만 남기고 제거

In [8]:
def text_cleaning(texts): # 수정완료
    # 한국어를 제외한 글자를 제거하는 패턴.
    corpus = []
    for i in range(0, len(texts)):
        sentences = re.sub("[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]", " ", str(texts[i])) #특수문자를 제거하는 패턴
        sentences = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 |0-9|]", " ", sentences)
        sentences = re.compile('[|ㄱ-ㅎ|ㅏ-ㅣ]+').sub(' ', sentences) # 자음, 모음만 있는 경우

        sentences = re.sub(r"^\s+", '', sentences) #remove space from start
        sentences = re.sub(r'\s+$', '', sentences) #remove space from the end

        sentences = re.sub(r'\s+', ' ', sentences) #remove extra space

        corpus.append(sentences)
    
    return corpus

In [9]:
pre_que = text_cleaning(que)
pre_ans = text_cleaning(ans)

# df에 업데이트
df['question_clean'] = pre_que
df['answer_clean'] = pre_ans

print(len(pre_que), len(pre_ans))

95190 95190


### hanspell 맞춤법

In [ ]:
# que
def hanspell_(texts):
    spell_check = []
    for line in tqdm(texts):
        spell = spell_checker.check(line).checked
        spell_check.append(spell)
    return spell_check

spell_check_question = hanspell_(pre_que)

In [ ]:
# 확인
print(spell_check_question[:5])

# df에 업데이트
df['question_spellcheck'] = spell_check_question

In [ ]:
# ans
def hanspell_(texts):
    spell_check = []
    for line in tqdm(texts):
        spell = spell_checker.check(line).checked
        spell_check.append(spell)
    return spell_check

spell_check_answer = hanspell_(pre_ans)

In [ ]:
# 확인
print(spell_check_answer[:5])

# df에 업데이트
df['answer_spellcheck'] = spell_check_answer

## 내보내기

In [ ]:
df.to_csv(save_path+'qna_table_hanspell.csv', index=False)